In this notebook a boosted tree ensemble model will be trained using Python's library xgboost for trying to predict the sale prices of the houses.

In [ ]:
import json
import os
import pickle

import numpy as np
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.preprocessing import StandardScaler

# 1. Load data

In [2]:
dataset = pd.read_csv('encoded_train_dataset.csv', sep = ',')

In [3]:
dataset

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,SalePrice
0,1,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,5.0,1.0,4.0,0.0,2.0,3.0,3.0,2.0,0.0,208500.0
1,2,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,5.0,1.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0,181500.0
2,3,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0,223500.0
3,4,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,4.0,1.0,4.0,4.0,1.0,3.0,3.0,2.0,0.0,140000.0
4,5,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,0.0,...,5.0,1.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0,175000.0
1456,1457,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,163.0,...,3.0,1.0,3.0,3.0,1.0,3.0,3.0,2.0,0.0,210000.0
1457,1458,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,0.0,...,5.0,1.0,4.0,4.0,2.0,3.0,3.0,2.0,0.0,266500.0
1458,1459,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,1029.0,...,4.0,1.0,4.0,0.0,1.0,3.0,3.0,2.0,0.0,142125.0


# 2. Train the model and check accuracy

The XGBRegressor algorithm chosen is trained  and the error metric was the negative root mean squared error.  
The model hyperparameters are optimized using Optuna's Python package.

In [4]:
X = dataset.drop(columns = ['Id', 'SalePrice'])
y = dataset['SalePrice'].values

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
def objective(trial):
    """Objective function.

    This function contains the objective function that Optuna will optimize. The chosen objective function is the maximum of the mean and median of the root mean squared errors generated by training the model using
    Leave One Out Cross Validation (LOOCV). The reason for not taking the mean of these errors directly is to prevent the hyperparameter optimizer from favouring those hyperparameters which yield particularly good
    scores only on specific folds.

    Args: 
        trial: optuna.trial.Trial object with a combination of hyperparameters for which the model will be trained and evaluated.

    Returns:
        float with the value of the objective function for the specific set of hyperparameters passed to the function in trial.
    """

    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log = True),  # L2 regularization term
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log = True),   # L1 regularization term
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),  # Sampling ratio
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),  # Feature selection
        'max_depth': trial.suggest_int('max_depth', 3, 9),  # Tree depth
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0),  # Learning rate
        'n_estimators': trial.suggest_int('n_estimators', 50, 500)  # Number of trees
    }

    # Train the XGBoost model using the suggested hyperparameters
    model = xgb.XGBRegressor(**param)
    cv = LeaveOneOut()

    # Calculate the RMSE (Root Mean Squared Error) as the evaluation metric
    scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)

    return np.max([-scores.mean(), -np.median(scores)]) # Objective functions

An Optuna study is created to optimize the hyperparameters of the xgboost model. Beware that this hyperparameter optimization took over a week to be executed due to the thoroughness of the optimization. If someone wants to execute a similar hyperparameter optimization for this data, I suggest to change the LOOCV for k-fold cross validation, with k a small number.

In [ ]:
# Create a study object and optimize it using Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-11-04 06:58:55,890] A new study created in memory with name: no-name-e0d8c883-6b27-416b-a1e5-0c639b178173
Best trial: 0. Best value: 26884.3:   1%|          | 1/100 [12:44:55<1262:07:04, 45895.20s/it]

[I 2024-11-04 19:43:51,083] Trial 0 finished with value: 26884.30034781678 and parameters: {'booster': 'dart', 'lambda': 2.5524695169150674e-08, 'alpha': 1.2216703955604143e-06, 'subsample': 0.8355329991357172, 'colsample_bytree': 0.4433401593214628, 'max_depth': 8, 'learning_rate': 0.8586224687091836, 'n_estimators': 440}. Best is trial 0 with value: 26884.30034781678.


Best trial: 0. Best value: 26884.3:   2%|▏         | 2/100 [21:14:22<1003:55:02, 36878.60s/it]

[I 2024-11-05 04:13:18,061] Trial 1 finished with value: 55949.427905607874 and parameters: {'booster': 'dart', 'lambda': 0.12395017653752258, 'alpha': 0.0016539847900046132, 'subsample': 0.422263871492796, 'colsample_bytree': 0.596699713693659, 'max_depth': 6, 'learning_rate': 0.7994351302035386, 'n_estimators': 359}. Best is trial 0 with value: 26884.30034781678.


Best trial: 2. Best value: 18686.7:   3%|▎         | 3/100 [21:16:11<541:17:12, 20088.99s/it] 

...

[I 2024-11-10 16:57:11,428] Trial 70 finished with value: 15884.479666095891 and parameters: {'booster': 'dart', 'lambda': 4.862633767950272e-06, 'alpha': 5.448396271157472e-07, 'subsample': 0.5253552293265782, 'colsample_bytree': 0.36499060140749406, 'max_depth': 5, 'learning_rate': 0.13720814830167455, 'n_estimators': 219}. Best is trial 46 with value: 14576.526723030822.


Best trial: 46. Best value: 14576.5:  72%|███████▏  | 72/100 [160:15:52<131:05:54, 16855.50s/it]

[I 2024-11-10 23:14:47,978] Trial 71 finished with value: 14683.22341609589 and parameters: {'booster': 'dart', 'lambda': 5.790516466149337e-07, 'alpha': 7.414630012930704e-06, 'subsample': 0.4729489389376773, 'colsample_bytree': 0.4917472549663895, 'max_depth': 5, 'learning_rate': 0.04137766853806776, 'n_estimators': 310}. Best is trial 46 with value: 14576.526723030822.


Best trial: 72. Best value: 14324.4:  73%|███████▎  | 73/100 [166:42:49<140:43:53, 18764.21s/it]

[I 2024-11-11 05:41:45,833] Trial 72 finished with value: 14324.352622003425 and parameters: {'booster': 'dart', 'lambda': 1.2246330866896723e-07, 'alpha': 8.340698586854568e-06, 'subsample': 0.651775221528867, 'colsample_bytree': 0.43749952717182383, 'max_depth': 5, 'learning_rate': 0.05486520178251461, 'n_estimators': 314}. Best is trial 72 with value: 14324.352622003425.


In [15]:
# Print the best hyperparameters and the corresponding RMSE
print(f"Best hyperparameters: {study.best_params}")
print(f"Best MSE: {study.best_value}")

Best hyperparameters: {'booster': 'dart', 'lambda': 1.2246330866896723e-07, 'alpha': 8.340698586854568e-06, 'subsample': 0.651775221528867, 'colsample_bytree': 0.43749952717182383, 'max_depth': 5, 'learning_rate': 0.05486520178251461, 'n_estimators': 314}
Best MSE: 14324.352622003425


Best hyperparameters are saved in a JSON file just to make sure they are not lost in case of any problem with the trained final model.

In [20]:
with open("xgboost_best_params.json", "w") as outfile: 
    json.dump(study.best_params, outfile)

# 3. Fit the model

The model is fitted with the optimized hyperparameters.

In [21]:
xg_reg = xgb.XGBRegressor(**study.best_params)

In [22]:
xg_reg.fit(X,y)

XGBRegressor(alpha=8.340698586854568e-06, base_score=None, booster='dart',
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.43749952717182383, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             lambda=1.2246330866896723e-07, learning_rate=0.05486520178251461,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=314, n_jobs=None, ...)

# 4. Save the model

The fitted model is saved using xgboost's model built-in method.

In [27]:
if not os.path.exists('./models/'):
    os.makedirs('./models/')
    
xg_reg.save_model('./models/xgboost_reg.json')